###Imports

In [ ]:
!pip install Unidecode
!pip install hdbscan
import pickle
import unidecode
import hdbscan
import re
import nltk
from nltk import FreqDist
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import string
from string import punctuation
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import umap

###Funções

In [ ]:
def grava(obj, path, filename):
	pkl_file = open(path + filename, 'wb')
	pickle.dump(obj, pkl_file)
	pkl_file.close()

def abre(path, filename):
	pkl_file = open(path + filename, 'rb')
	obj = pickle.load(pkl_file)
	pkl_file.close()
	return obj

def get_nome_representacao_do_grupo_v4(df2,qtd_palavras,unidades):
	percentual_pra_manter_palavra_na_representacao = 0.50
	#pega cada palavra e ve as que mais se repetem nas sentences
	#fica com aquelas que estao em mais do que X% das sentences
	sentences = [sent for sent in df2['xprod']]
	if len(set(sentences)) == 1: #ou seja, todos os itens sao iguais.
		representacao_grupo = [word for word in set(sentences)][0].split()
	else:
		palavras_series = df2['xprod'].str.split()
		palavras_series = palavras_series.apply(lambda x: x[:qtd_palavras])
		contagem_palavras_nas_sentences = {}
		palav = [item for sublist in palavras_series for item in sublist]
		palavras = sorted( set(palav), key=palav.index) #pra preservar a ordem em que as palavras aparecem (set puro coloca em ordem alfabetica)
		for palavra in palavras:
			contagem_palavras_nas_sentences[palavra] = (palavras_series.apply(lambda x: palavra in x)).sum()
	
		contagem_palavras_nas_sentences = pd.Series(contagem_palavras_nas_sentences)
		contagem_palavras_nas_sentences = contagem_palavras_nas_sentences / len(df2)
		contagem_palavras_nas_sentences = contagem_palavras_nas_sentences[contagem_palavras_nas_sentences > percentual_pra_manter_palavra_na_representacao]
		representacao_grupo = list(contagem_palavras_nas_sentences.index)
	#reordenacao:
	primeiras_palavras = [word for word in representacao_grupo if ((not word.isdigit()) and word not in unidades)]
	meio_palavras = [word for word in representacao_grupo if word.isdigit()]
	ultimas_palavras = [word for word in representacao_grupo if word in unidades]
	
  # intercala numeros e unidades:
	if len(meio_palavras) == len(ultimas_palavras):
		result = [None]*(len(meio_palavras)+len(ultimas_palavras))
		result[::2] = meio_palavras
		result[1::2] = ultimas_palavras
		meio_palavras = result
		ultimas_palavras = []
	else:
		# if 'x' in representacao_grupo:
		if (('x' in ultimas_palavras) and (len(meio_palavras) > 0)):
			ultimas_palavras = [word for word in ultimas_palavras if word != 'x'] #retira o 'x', vai inserir abaixo:
			meio_palavras.insert(1,'x') #insere o 'x' apos o 1o numero
	if len(meio_palavras) == 0:
		representacao_grupo = primeiras_palavras #daih nao coloca unidades
	else:
		representacao_grupo = primeiras_palavras + meio_palavras + ultimas_palavras
	return representacao_grupo

def print_exemplos_grupos_v2_aleatorio(df,grupos,grupox,cols,qtd_palavras,unidades):
	#inicio = np.random.randint(len(grupos)-qtd_grupos_mostrar)
	#fim = inicio + qtd_grupos_mostrar
	for grupo in grupos:
		df_mostrar = df[df[grupox] == grupo]
		print('\nGrupo:',grupo,'len:',len(df_mostrar))
		print(get_nome_representacao_do_grupo_v4(df2=df[df[grupox]==grupo],qtd_palavras=qtd_palavras,unidades=unidades))
		print(df_mostrar[cols])

def calc_preco(df, grupos, grupox):
	cols = ['NCM','xprod', 'vuncom']
	df_list_grupos = []

	groups = []
	group_names = []
	media = []
	mediana = []
	media_saneada = []
	precos_max = []
	precos_min = []
	reindex = {}
	
	for i, grupo in enumerate(grupos):
		df_grupo = df[df[grupox] == grupo]
		df_list_grupos.append(df_grupo)	
		
		#S = E – M sobrepreço estimado - referencial mercado
		media.append(round(df_grupo['vuncom'].mean(),3))
		mediana.append(round(df_grupo['vuncom'].median(),3))
		
		nova_media, preco_max, preco_min = calc_media_saneada(df_grupo)
		
		media_saneada.append(nova_media)
		precos_max.append(preco_max)
		precos_min.append(preco_min)
	
		descricao = get_nome_representacao_do_grupo_v4(df2=df[df[grupox]==grupo],qtd_palavras=qtd_palavras,unidades=unidades)
		descricao = ' '.join(descricao)
		group_names.append(descricao)
		reindex[grupo] = i+1
		groups.append(i+1)
	
	df_precos = pd.DataFrame(zip(groups, group_names, media, mediana, precos_max, precos_min, media_saneada),\
	                         columns=["id","Descricao", "Media", "Mediana", "Max", "Min", "Media Saneada"])
	
	df_precos = df_precos.round(2)
	df_precos['Media'] = df_precos['Media'].astype(str).str.replace('.',",")  
	df_precos['Mediana'] = df_precos['Mediana'].astype(str).str.replace('.',",")  
	df_precos['Max'] = df_precos['Max'].astype(str).str.replace('.',",")  
	df_precos['Min'] = df_precos['Min'].astype(str).str.replace('.',",")  
	df_precos['Media Saneada'] = df_precos['Media Saneada'].astype(str).str.replace('.',",")
	
	df_precos.set_index('id', inplace=True)
	
	return df_precos, df_list_grupos, reindex

def calc_media_saneada(df):
	preco = df['vuncom'].dropna()
	preco_max = max(preco)
	preco_min = min(preco)
	preco_mean = preco.mean()
	LS = preco_mean + preco.std()
	LI = preco_mean - preco.std()		
	CV = abs(preco.std() / preco_mean)
	
	if CV > 0.25:
		precos = preco[preco.between(LI, LS)]
		return (precos.mean(), preco_max, preco_min)
	else:
		return (preco_mean, preco_max, preco_min)

def descricoes_grupos(df, grupos, grupox):
	groups = []
	descrs = []
	for i, grupo in enumerate(grupos):
		df_grupo = df[df[grupox] == grupo]
		descr_unicas = df_grupo['xprod'].unique()
		for descr in descr_unicas:
			groups.append(i+1)
			descrs.append(descr)
	df_descrs_unicas = pd.DataFrame(zip(groups, descrs), columns=["Grupo", "Descricao"])
	return df_descrs_unicas

def save_groups(dfs):
	for i, df in enumerate(dfs):
		df.to_excel(f'grupo{i}.xlsx')


###Variaveis

In [ ]:
qtd_dimensoes = 200
qtd_dimensoes_umap = 10
qtd_palavras = 10
tamanho_minimo_pra_formar_grupo = 35
min_samples = 1
quantile_a_retirar_outliers_dbscan = 0.99
similarity_minima_pra_juntar_grupos = 0.65
percentual_primeira_palavra_igual_pra_considerar_grupo_homogeneo = 0.70
quantile_a_retirar_quantidade_palavras_diferentes_no_grupo = 0.95
quantile_a_retirar_numeros_diferentes_no_grupo = 0.95
similarity_minima_pra_encaixar_itens_excluidos_no_final = 0.85


unidades = ['x','mm','m','cm','ml','g','mg','kg','unidade','unidades','polegada','polegadas','grama','gramas','gb','mb','l','lr','lt', 'ltr', 'litro', 'ltrs', 'litros','mts','un','mgml','w','hz','v','gr','lts','lonas','cores','mcg']
path_drive = 'drive/My Drive/Residencia/'

###Descriptions Cleaning

In [ ]:
df = pd.read_csv('produtos.csv')
df_original = df.copy()
################
#Análise de ceans nulos 

#print(df[df['cean'] == 'sem gtin'].count())
#print(df.isnull().count())
#cean: 441785(total)-373477(nao nulos) = 68308 nulos + 269628 'sem gtin' = 337936 : 76% inuteis

print(len(df))
print(collections.Counter(df['ncm'].values.tolist()).most_common(5))

#filtra df para apenas mais comuns (combustiveis)
most_common = [ncm for ncm, qtd in collections.Counter(df['ncm'].values.tolist()).most_common(2)]

df = df[df['ncm'].apply(lambda ncm: ncm in most_common)]

#df['xprod'].iloc[np.random.randint(len(df),size=50)]
#df['xprod'].iloc[np.random.randint(len(df),size=50)].apply(len)


496309
[(27101921, 12925), (27101259, 10918), (87089990, 9228), (30049099, 8619), (39174090, 7610)]


In [ ]:
nltk.download('stopwords')

pt_stopwords = set(nltk.corpus.stopwords.words("portuguese"))
stopwords = set(list(punctuation))
stopwords.add('oleo')
stopwords.add('petrobras')
stopwords.add('bs')
stopwords.add('br')
stopwords.add('tipo')
pt_stopwords.update(stopwords)

#limpa sentencas retirando stopwords, pontuacao e deixa minusculo.
df['xprod'] = [ ' '.join([word.lower() for word in descr.split() if word.lower() not in pt_stopwords]) for descr in df['xprod'].astype(str)]
#insere espaco apos / e -, pra no final nao ficar palavras assim: csolucao, ptexto (originais eram c/solucao, p-texto)
df['xprod'] = df['xprod'].apply(lambda descr: re.sub(r'/|-',r' ',descr))
#retira pontuacao, /, etc:
df['xprod'] = df['xprod'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
#insere espaco apos numero e letra (separa unidades de medida:) ex.: 500ml vs 100ml vs 500mg
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'(\d{1})(\D)',r'\1 \2',x))
#insere espaco apos letra e numero ex.:c100 pc50
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'(\D{1})(\d)',r'\1 \2',x))
#apaga caracteres de tamanho 1
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r'\b[a-zA-Z]\b',r'',x))
#retira espacos duplicados
df['xprod'] = df['xprod'].apply(lambda x: re.sub(r' +',r' ', x))
#retira espaco no inicio da frase
df['xprod'] = df['xprod'].apply(lambda x: x.strip())
#retira acentos:
df['xprod'] = df['xprod'].apply(lambda x: unidecode.unidecode(x))
# remove zeros a esquerda de numeros (02 litros, 05, etc.)
df['xprod'] = df['xprod'].apply(lambda x: ' '.join([word.lstrip('0') for word in x.split()] ) )
# remove 'x', pra não diferenciar pneu 275 80 de 275 x 80:
df['xprod'] = df['xprod'].apply(lambda x: ' '.join([word for word in x.split() if word is not 'x']))
#se primeira palavra for numero, joga pro final (caso de numeros de referencia que colocam no inicio)
df['xprod'] = df['xprod'].apply(lambda x: ' '.join(x.split()[1:] + [x.split()[0]]) if ((len(x) > 1) and (x.split()[0].isdigit()) ) else x)
#limpa sentencas retirando stopwords
df['xprod'] = [ ' '.join([word for word in descr.split() if word not in pt_stopwords]) for descr in df['xprod'].astype(str)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#df['xprod'].iloc[np.random.randint(len(df),size=50)]

###Pre processing

In [ ]:
sentences = [descr.split()[:qtd_palavras] for descr in df['xprod']]

####Aplicando pesos

#####Fasttext

In [ ]:
'''
model = FastText(size=qtd_dimensoes, min_count=1)  
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)
'''
'''
#Conversao word embeddings to sentence embedding, com pesos
#dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:
doc_vectors = {}
for number, sent in enumerate(tqdm(sentences)):
    if len(sent) == 0:
        doc_vectors[number] = np.zeros(qtd_dimensoes,)
    elif len(sent) == 1:
        doc_vectors[number] = model[sent[0]]
    elif len(sent) > 1:
        pesos = np.array(range(len(sent))[::]) + 1
        # agora com pesos 1/x - tem que ser na ordem 1,2,..., os menores numeros dao maiores pesos
        pesos = 1 / pesos 
        media = []
        divisao = 0
        counter = 0
        for word in sent:
            if word.isdigit():
                #se eh digit, atribui peso NO 3/4 da faixa entre o peso da primeira e da ultima palavra. 
                #Mesmo peso pra todos os numeros, mais importante que palavras do fim, menos importante que palavras do inicio.
                media.append(model.wv[word] * ((pesos[0]+pesos[-1])*(1/4))) 
                divisao += ((pesos[0]+pesos[-1])*(1/4))
            else:
                media.append(model.wv[word] * pesos[counter])
                divisao += pesos[counter]
            counter += 1
        doc_vectors[number] = np.array(media).sum(axis=0) / divisao #media de tudo
'''

'\n#Conversao word embeddings to sentence embedding, com pesos\n#dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:\ndoc_vectors = {}\nfor number, sent in enumerate(tqdm(sentences)):\n    if len(sent) == 0:\n        doc_vectors[number] = np.zeros(qtd_dimensoes,)\n    elif len(sent) == 1:\n        doc_vectors[number] = model[sent[0]]\n    elif len(sent) > 1:\n        pesos = np.array(range(len(sent))[::]) + 1\n        # agora com pesos 1/x - tem que ser na ordem 1,2,..., os menores numeros dao maiores pesos\n        pesos = 1 / pesos \n        media = []\n        divisao = 0\n        counter = 0\n        for word in sent:\n            if word.isdigit():\n                #se eh digit, atribui peso NO 3/4 da faixa entre o peso da primeira e da ultima palavra. \n                #Mesmo peso pra todos os numeros, mais importante que palavras do fim, menos importante que palavras do inicio.\n                media.append(model.w

#####Word2vec

In [ ]:
model2 = Word2Vec(sentences,size=qtd_dimensoes, min_count=1, workers=-1)
doc_vectors2 = {}

for number, sent in enumerate(sentences):
    #agora dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:
    if len(sent) == 0:
        doc_vectors2[number] = np.zeros(qtd_dimensoes,)
    elif len(sent) == 1:
        doc_vectors2[number] = model2.wv[sent[0]]
    elif len(sent) > 1:
        pesos = np.array(range(len(sent))[::]) + 1
        pesos = 1 / pesos
        media = []
        divisao = 0
        counter = 0
        for word in sent:
            #### AGORA O MODEL EH W2V E O PESO EH DOBRADO PRA DIGITS:
            if word.isdigit():
                media.append(model2.wv[word] * ((pesos[0]+pesos[-1])*(1/2)) )
                divisao += ((pesos[0]+pesos[-1])*(1/2))
            else:
                media.append(model2.wv[word] * pesos[counter])
                divisao += pesos[counter]
            counter += 1
        doc_vectors2[number] = np.array(media).sum(axis=0) / divisao #media de tudo

In [ ]:
doc_vectors2 = pd.DataFrame(doc_vectors2).T
doc_vectors3 = doc_vectors2
doc_vectors2 = doc_vectors2.set_index(df.index) 

scaler = StandardScaler()
doc_vectors_std_df = pd.DataFrame(scaler.fit_transform(doc_vectors2),index=doc_vectors2.index,columns=doc_vectors2.columns)

####Redução de dimensão

In [ ]:
#n_neighbors=20
umap_redux = umap.UMAP(n_components=qtd_dimensoes_umap, random_state=999, metric='cosine')
umap_redux.fit(doc_vectors_std_df)

doc_vectors_std_df_umap = umap_redux.transform(X=doc_vectors_std_df)

###Clustering

In [ ]:
clustering = hdbscan.HDBSCAN(min_cluster_size=tamanho_minimo_pra_formar_grupo,min_samples=min_samples,prediction_data=True,core_dist_n_jobs=-1)\
              .fit(doc_vectors_std_df_umap)
#clustering = abre(path_drive, 'HDBSCAN_combust.pkl')

df['grupo'] = clustering.labels_

In [ ]:
threshold = pd.Series(clustering.outlier_scores_).quantile(quantile_a_retirar_outliers_dbscan)
outliers = np.where(clustering.outlier_scores_ > threshold)[0]
df.iloc[outliers,df.columns.get_loc('grupo')] = -2

In [ ]:
grupos = np.unique(clustering.labels_)
grupos = [grupo for grupo in grupos if grupo >= 0]

print('qtd de grupos:',len(grupos))

qtd de grupos: 25


###Cluster Cleaning

In [ ]:
#########################################
# EXCLUI sentences outliers pelo texto pelos exemplars
# FAZ cosine distance pra cada sentence dentro dos grupos
#########################################

#pontos mais representativos de cada grupo (como hdbscan aceita formato aleatorio de cada cluster, nao tem como usar centroide)
exemplars = []
for exemplar in tqdm(clustering.exemplars_):
    exemplars.append(np.mean(exemplar,axis=0))
exemplars_df = pd.DataFrame(exemplars,index=range(len(grupos)))

map_grupos_exemplars = {}
df_temp = pd.DataFrame(columns=['sims'])

for grupo in tqdm(grupos[:]):
    df2 = df[df['grupo'] == grupo]
    indexes = df2.index
    grupo_vectors = pd.DataFrame(doc_vectors_std_df_umap,index=df.index).loc[indexes]
    
    grupo_do_exemplar = pd.Series(cosine_similarity(grupo_vectors.mean(axis=0).values.reshape(1,-1),exemplars_df)[0]).sort_values(ascending=False).index[0]
    map_grupos_exemplars[grupo] = grupo_do_exemplar

    sims = cosine_similarity(grupo_vectors,exemplars[grupo_do_exemplar].reshape(1,-1))

    df2['sims'] = sims
    df_temp = df_temp.append(df2[['sims']])

#passa resultados pro df principal:
df['sims'] = df_temp
df['sims'] = df['sims'].replace(np.nan,-1)

#retira quem tem similaridade negativa - eh um bom parametro.
df['grupo2'] = np.where(df['sims'] < 0, -1, df['grupo'])

grupos = df['grupo2'].unique()
grupos = [grupo for grupo in grupos if grupo >= 0]

print('qtd de grupos:',len(grupos))

In [ ]:
##########################################
# Se a 1a palavra nao for a mesma em X% do grupo, exclui o grupo, eh muito heterogeneo
##########################################
df['grupo3'] = df['grupo2']

grupos = sorted(df['grupo3'].unique())
grupos = [grupo for grupo in grupos if grupo >=0] #tirar os -1, -2, etc.

grupos_homogeneos = []

for grupo in tqdm(grupos):
    df2 = df[df['grupo3'] == grupo]
    if len(df2) > 0:
        if ( df2['xprod'].apply(lambda x: x.split()[0] if (len(x.split()) > 0) else np.random.random()).value_counts().iloc[0] / len(df2) ) > percentual_primeira_palavra_igual_pra_considerar_grupo_homogeneo:
            grupos_homogeneos.append(grupo)

df['grupo4'] = df['grupo3'].isin(grupos_homogeneos)
df['grupo4'] = np.where(df['grupo4'], df['grupo3'], -1)

grupos = sorted(df['grupo4'].unique())
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

100%|██████████| 25/25 [00:00<00:00, 230.49it/s]

qtd de grupos: 25


In [ ]:
############################################################################
# EXCLUI GRUPOS NAO HOMOGENEOS - pela contagem de palavras diferentes! 
############################################################################

df['qtd_palavras_diferentes'] = df['xprod'].apply(lambda x: len(set( [item for sublist in [sent.split()[:qtd_palavras] for sent in x] for item in sublist] )))
qtd_palavras_por_grupo = df.groupby('grupo4')['qtd_palavras_diferentes'].median()
qtd_palavras_por_grupo = qtd_palavras_por_grupo.sort_values()
qtd_max_palavras_diferentes_no_grupo = int(qtd_palavras_por_grupo.quantile(quantile_a_retirar_quantidade_palavras_diferentes_no_grupo))
print('Quantidade maxima de palavras diferentes aceita por grupo:', qtd_max_palavras_diferentes_no_grupo)

df['qtd_median_palavras_dif_grupo'] = df['grupo4'].map(qtd_palavras_por_grupo)

df['grupo5'] = np.where(df['qtd_median_palavras_dif_grupo'] > qtd_max_palavras_diferentes_no_grupo, -1, df['grupo4'])

grupos = df['grupo5'].unique()
grupos = [grupo for grupo in grupos if grupo >=0]

print('qtd de grupos:',len(grupos))

Quantidade maxima de palavras diferentes aceita por grupo: 17
qtd de grupos: 23


In [ ]:
df['grupo6'] = df['grupo5']
############################################################################
# EXCLUI GRUPOS que tem MUITOS NUMEROS DIFERENTES NAS SENTENCES
############################################################################

df['qtd_numeros_diferentes'] = df['xprod'].apply(lambda x: len(set( [item for sublist in [sent.split()[:qtd_palavras] for sent in x] for item in sublist if item.isdigit()   ] ) ))
df['qtd_numeros_diferentes'] = np.where(df['qtd_numeros_diferentes'] == 0, 0, df['qtd_numeros_diferentes']-1)
qtd_numeros_por_grupo = df.groupby('grupo4')['qtd_numeros_diferentes'].median()
qtd_numeros_por_grupo = qtd_numeros_por_grupo[qtd_numeros_por_grupo > 0]
qtd_numeros_por_grupo = qtd_numeros_por_grupo.sort_values()
qtd_max_numeros_diferentes_no_grupo = int(qtd_numeros_por_grupo.quantile(quantile_a_retirar_numeros_diferentes_no_grupo))
print('Quantidade maxima de numeros diferentes aceita por grupo:', qtd_max_numeros_diferentes_no_grupo)

df['qtd_median_numeros_dif_grupo'] = df['grupo4'].map(qtd_numeros_por_grupo)

df['grupo7'] = np.where(df['qtd_median_numeros_dif_grupo'] > qtd_max_numeros_diferentes_no_grupo, -1, df['grupo6'])

grupos = df['grupo7'].unique()
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

Quantidade maxima de numeros diferentes aceita por grupo: 3
qtd de grupos: 23


In [ ]:
doc_vectors3 = doc_vectors3.set_index(df.index) 
doc_vectors_grupos = {}

for grupo in tqdm(grupos):
    indices = df[df['grupo7'] == grupo].index
    doc_vectors_grupos[grupo] = doc_vectors3.loc[indices]
    doc_vectors_grupos[grupo] = doc_vectors_grupos[grupo].mean(axis=0)   

doc_vectors_grupos = pd.DataFrame(doc_vectors_grupos).T
#usa o scaler original:
doc_vectors_grupos_std = pd.DataFrame(scaler.transform(doc_vectors_grupos),index=doc_vectors_grupos.index,columns=doc_vectors_grupos.columns)

grupos_similarities = cosine_similarity(doc_vectors_grupos_std)
grupos_similarities = pd.DataFrame(grupos_similarities,index=doc_vectors_grupos.index,columns=doc_vectors_grupos.index)

#junta os grupos:
grupos_similares = []
for grupo in tqdm(grupos_similarities):
    agrupar_df = grupos_similarities[grupo].sort_values(ascending=False)
    agrupar_df = agrupar_df[agrupar_df >= similarity_minima_pra_juntar_grupos]
    grupos_similares.append(list(agrupar_df.index))

100%|██████████| 23/23 [00:00<00:00, 519.22it/s]


In [ ]:
novo_grupo = 0
mapeamento_grupos = {}
for mini_grupo in tqdm(grupos_similares):
    if len(mini_grupo) == 1:
        mapeamento_grupos[mini_grupo[0]] = novo_grupo
    else:
        for grupo in mini_grupo:
            if grupo not in mapeamento_grupos.keys():
                for mini_grupo2 in grupos_similares:
                    if grupo in mini_grupo2:
                        mapeamento_grupos[grupo] = novo_grupo
                        for grupo2 in mini_grupo2:
                            if grupo2 not in mapeamento_grupos.keys():
                                mapeamento_grupos[grupo2] = novo_grupo
    novo_grupo += 1

df['grupo9'] = df['grupo7'].map(mapeamento_grupos)
df['grupo9'] = df['grupo9'].fillna(-1)
df['grupo9'] = df['grupo9'].astype(int)

grupos = df['grupo9'].unique()
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

100%|██████████| 23/23 [00:00<00:00, 35689.60it/s]

qtd de grupos: 6


In [ ]:
###############################################
# Repassar as sentences do grupo -1 e tentar encaixá-las nos grupos formados, só com grande certeza.
###############################################

excluidos_index =  df[df['grupo9'] == -1].index
incluidos_index =  df[df['grupo9'] >= 0].index

doc_vectors_excluidos = doc_vectors2.loc[excluidos_index]
doc_vectors_incluidos = doc_vectors2.loc[incluidos_index]

doc_vectors_grupos_finais = {}
for grupo in tqdm(grupos):
    df2 = df[df['grupo9'] == grupo]
    doc_vectors_grupos_finais[grupo] = doc_vectors_incluidos.loc[df2.index]
    doc_vectors_grupos_finais[grupo] = doc_vectors_grupos_finais[grupo].values.mean(axis=0)

# index aqui jah eh o numero certo dos grupos:
doc_vectors_grupos_finais = pd.DataFrame(doc_vectors_grupos_finais).T
compara = cosine_similarity(doc_vectors_excluidos.loc[excluidos_index],doc_vectors_grupos_finais.values)
compara = pd.DataFrame(compara,index=excluidos_index, columns=grupos)
similarity_do_grupo_mais_parecido = compara.max(axis=1)
grupo_mais_parecido = compara.idxmax(axis=1)

encaixar_excluidos = pd.Series( np.where(similarity_do_grupo_mais_parecido >= similarity_minima_pra_encaixar_itens_excluidos_no_final, grupo_mais_parecido, -1), index= similarity_do_grupo_mais_parecido.index)
df['grupo10'] = encaixar_excluidos
df['grupo10'] = df['grupo10'].fillna(-1)
df['grupo10'] = np.where(df['grupo10'] == -1, df['grupo9'], df['grupo10'])

grupos = df['grupo10'].unique()
grupos = [grupo for grupo in grupos if grupo >=0] 

print('qtd de grupos:',len(grupos))

100%|██████████| 6/6 [00:00<00:00, 237.04it/s]

qtd de grupos: 6


###Resultados

In [ ]:
#grupos_similares
#grupos_similarities

In [ ]:
print_exemplos_grupos_v2_aleatorio(df=df,grupos=grupos,grupox='grupo10',cols=['xprod', 'vuncom'],qtd_palavras=qtd_palavras,unidades=unidades)


Grupo: 0.0 len: 6455
['diesel', '10']
                      xprod    vuncom
11                diesel 10  3.930000
51                diesel 10  3.750000
152               diesel 10  3.750000
177               diesel 10  3.750000
595               diesel 10  3.532000
...                     ...       ...
494528            diesel 10  3.320000
494839  diesel aditivado 10  3.310007
494842  diesel aditivado 10  3.309999
494850            diesel 10  3.330000
495960            diesel 10  2.990000

[6455 rows x 2 columns]

Grupo: 1.0 len: 4809
['diesel', '500']
                   xprod  vuncom
12            diesel 500    3.83
49            diesel 500    3.86
304           diesel 500    3.82
374           diesel 500    3.90
403           diesel 500    3.49
...                  ...     ...
494847        diesel 500    3.28
496243  diesel 500 comum    3.17
496248  diesel 500 comum    3.17
496249  diesel 500 comum    3.17
496250  diesel 500 comum    3.17

[4809 rows x 2 columns]

Grupo: 2.0 len: 10

In [ ]:
values = [-1, -2] #verificar os que ficaram de fora (-1) e foram considerados outliers (-2)
out_inicio = df[df.grupo.isin(values)]
out_fim = df[df['grupo10']==-1]
resu = (len(out_inicio)+len(out_fim))/(len(df['ncm'].values.tolist()))*100

print(f'Ficaram fora no início: {len(out_inicio)}')
print(f'Ficaram fora no fim: {len(out_fim)}')
print(f'Total de registros de fora (%): {resu}')

Ficaram fora no início: 272
Ficaram fora no fim: 329
Total de registros de fora (%): 2.520655957723441


In [ ]:
#out_inicio.to_excel('ficaram_fora_inicio.xlsx')
#out_fim.to_excel('ficaram_fora_fim.xlsx')

In [ ]:
df_precos, df_list_grupos, reindex = calc_preco(df=df,grupos=grupos,grupox='grupo10')
#save_groups(df_list_grupos)
df_precos.to_csv('precos_classes_combustiveis.csv')
df_precos

,Descricao,Media,Mediana,Max,Min,Media Saneada
id,,,,,,
1,diesel 10,"3,82","3,89","5,68","2,67","3,82"
2,diesel 500,"4,48","3,75","3854,0","2,51","3,68"
3,gasolina comum,"4,48","4,59","5,09","3,17","4,48"
4,diesel comum,"3,69","3,77","4,1","2,78","3,69"
5,gasolina grid,"4,54","4,6","4,94","3,84","4,54"
6,etanol comum,"3,52","3,65","3,81","3,0","3,51"


In [ ]:
df_descrs_unicas = descricoes_grupos(df=df,grupos=grupos,grupox='grupo10')
df_descrs_unicas

,Grupo,Descricao
0,1,diesel 10
1,1,diesel 10 comum
2,1,diesel 10 comun
3,1,diesel 10 lt
4,1,diesel comum 10
5,1,diesel 10 ds 10
6,1,diesel aditivado 10
7,1,10
8,1,diesel 10 aditivado grid
9,1,diesel 10 grid


In [ ]:
'''
df_original['grupo10'] = df['grupo10']
df_original['grupo10'] = df_original['grupo10'].fillna(-1)

df_descrs_unicas_orig = descricoes_grupos(df=df_original,grupos=grupos,grupox='grupo10')
#df_descrs_unicas_orig
'''

In [ ]:
grava(clustering, path_drive, 'HDBSCAN_combustiveis.pkl')
grava(df, path_drive, 'produtos_treinamento.pkl')
grava(doc_vectors2, path_drive, 'doc2vec_combustiveis.pkl')
grava(model2, path_drive, 'model_word2vec_combustiveis.pkl')

In [ ]:
grava(doc_vectors2, path_drive, 'doc2vec_combustiveis.pkl')

###Predição

In [ ]:
df_teste = pd.read_csv('produtos.csv')

most_common = [ncm for ncm, qtd in collections.Counter(df_teste['ncm'].values.tolist()).most_common(2)]

df_teste = df_teste[df_teste['ncm'].apply(lambda ncm: ncm in most_common)]
print(len(df_teste))
df_teste['grupo9'] = -1

23843


In [ ]:
nltk.download('stopwords')

pt_stopwords = set(nltk.corpus.stopwords.words("portuguese"))
stopwords = set(list(punctuation))
stopwords.add('oleo')
stopwords.add('petrobras')
stopwords.add('bs')
stopwords.add('br')
stopwords.add('tipo')
pt_stopwords.update(stopwords)

#limpa sentencas retirando stopwords, pontuacao e deixa minusculo.
df_teste['xprod'] = [ ' '.join([word.lower() for word in descr.split() if word.lower() not in pt_stopwords]) for descr in df_teste['xprod'].astype(str)]
#insere espaco apos / e -, pra no final nao ficar palavras assim: csolucao, ptexto (originais eram c/solucao, p-texto)
df_teste['xprod'] = df_teste['xprod'].apply(lambda descr: re.sub(r'/|-',r' ',descr))
#retira pontuacao, /, etc:
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
#insere espaco apos numero e letra (separa unidades de medida:) ex.: 500ml vs 100ml vs 500mg
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: re.sub(r'(\d{1})(\D)',r'\1 \2',x))
#insere espaco apos letra e numero ex.:c100 pc50
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: re.sub(r'(\D{1})(\d)',r'\1 \2',x))
#apaga caracteres de tamanho 1
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: re.sub(r'\b[a-zA-Z]\b',r'',x))
#retira espacos duplicados
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: re.sub(r' +',r' ', x))
#retira espaco no inicio da frase
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: x.strip())
#retira acentos:
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: unidecode.unidecode(x))
# remove zeros a esquerda de numeros (02 litros, 05, etc.)
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: ' '.join([word.lstrip('0') for word in x.split()] ) )
# remove 'x', pra não diferenciar pneu 275 80 de 275 x 80:
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: ' '.join([word for word in x.split() if word is not 'x']))
#se primeira palavra for numero, joga pro final (caso de numeros de referencia que colocam no inicio)
df_teste['xprod'] = df_teste['xprod'].apply(lambda x: ' '.join(x.split()[1:] + [x.split()[0]]) if ((len(x) > 1) and (x.split()[0].isdigit()) ) else x)
#limpa sentencas retirando stopwords
df_teste['xprod'] = [ ' '.join([word for word in descr.split() if word not in pt_stopwords]) for descr in df_teste['xprod'].astype(str)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sentences_teste = [descr.split()[:qtd_palavras] for descr in df_teste['xprod']]

#model2 = Word2Vec(sentences_teste,size=qtd_dimensoes, min_count=1, workers=-1)
#usa o model2 original de treino

In [ ]:
doc_vectors_teste = {}
for number, sent in enumerate(sentences_teste):
    #agora dando peso maior pras primeiras palavras, peso decrescente ateh o final, numeros com mesmo peso da primeira palavra:
    if len(sent) == 0:
        doc_vectors_teste[number] = np.zeros(qtd_dimensoes,)
    elif len(sent) == 1:
        doc_vectors_teste[number] = model2.wv[sent[0]]
    elif len(sent) > 1:
        pesos = np.array(range(len(sent))[::]) + 1
        pesos = 1 / pesos
        media = []
        divisao = 0
        counter = 0
        for word in sent:
            if word.isdigit():
                media.append(model2.wv[word] * ((pesos[0]+pesos[-1])*(1/2)) )
                divisao += ((pesos[0]+pesos[-1])*(1/2))
            else:
                media.append(model2.wv[word] * pesos[counter])
                divisao += pesos[counter]
            counter += 1
        doc_vectors_teste[number] = np.array(media).sum(axis=0) / divisao #media de tudo
doc_vectors_teste = pd.DataFrame(doc_vectors_teste).T
doc_vectors_teste = doc_vectors_teste.set_index(df_teste.index) 

In [ ]:
###############################################
# Repassar as sentences do grupo -1 e tentar encaixá-las nos grupos formados
###############################################

#excluidos_index =  df[df['grupoteste'] == -1].index
excluidos_index = doc_vectors_teste.index
incluidos_index =  df[df['grupo9'] >= 0].index

doc_vectors2 = abre(path_drive, 'word2vec_combustiveis.pkl')

doc_vectors_excluidos = doc_vectors_teste
doc_vectors_incluidos = doc_vectors2.loc[incluidos_index]

doc_vectors_grupos_finais = {}
for grupo in grupos:
    df2 = df[df['grupo9'] == grupo]
    doc_vectors_grupos_finais[grupo] = doc_vectors_incluidos.loc[df2.index]
    doc_vectors_grupos_finais[grupo] = doc_vectors_grupos_finais[grupo].values.mean(axis=0)

doc_vectors_grupos_finais = pd.DataFrame(doc_vectors_grupos_finais).T

compara = cosine_similarity(doc_vectors_excluidos,doc_vectors_grupos_finais.values)
compara = pd.DataFrame(compara,index=excluidos_index, columns=grupos)
similarity_do_grupo_mais_parecido = compara.max(axis=1)
grupo_mais_parecido = compara.idxmax(axis=1)

encaixar_excluidos = pd.Series( np.where(similarity_do_grupo_mais_parecido >= similarity_minima_pra_encaixar_itens_excluidos_no_final, grupo_mais_parecido, -1),\
                               index= similarity_do_grupo_mais_parecido.index)
df_teste['grupo10'] = encaixar_excluidos
df_teste['grupo10'] = df_teste['grupo10'].fillna(-1)
df_teste['grupo10'] = np.where(df_teste['grupo10'] == -1, df_teste['grupo9'], df_teste['grupo10'])

In [ ]:
print_exemplos_grupos_v2_aleatorio(df=df_teste,grupos=grupos,grupox='grupo10',cols=['xprod', 'vuncom'],qtd_palavras=qtd_palavras,unidades=unidades)

In [ ]:
cols_interesse = ['idnfe', 'dhemi', 'xnome', 'ncm', 'cean', 'ucom', 'qcom', 'xprod', 'vuncom', 'vprod', 'grupo10']
#reindex dos id grupos (de numeros aleatórios para sequenciais)
df_teste.reset_index(inplace=True)
df_teste['grupo10'] = df_teste['grupo10'].map(reindex)
df_teste = df_teste[df_teste['grupo10'] >= 0]
df_teste = df_teste[cols_interesse]

df_teste = df_teste.round(2)
df_teste['grupo10'] = df_teste['grupo10'].astype(int)
df_teste['idnfe'] = df_teste['idnfe'].astype(str)
df_teste['qcom'] = df_teste['qcom'].astype(str).str.replace('.',",")
df_teste['vuncom'] = df_teste['vuncom'].astype(str).str.replace('.',",")
df_teste['vprod'] = df_teste['vprod'].astype(str).str.replace('.',",")

df_teste.rename(columns={'grupo10':'id', 'dhemi':'Data', 'ncm':'NCM', 'xnome':'Orgao', 'xprod':'Descricao do Produto',\
                         'xprod':'Descricao do Produto', 'cean':'Codigo de Barras', 'qcom':'Quantidade Comercial',\
                         'ucom':'Unidade de Comercialização', 'vuncom':'Valor Unitário de Comercialização', 'vprod':'Total'}, inplace=True)

df_teste.to_csv('produtos_cassificados_combustiveis.csv')

In [ ]:
#df_final = df.append(df_teste, ignore_index=True)